# 淘寶用戶行為分析

## 資料引入與清理過程

In [1]:
# Analyzing Tools
import pandas as pd
# import pandas_profiling
import numpy as np

# Ploting Tools
import matplotlib.pyplot as plt
import plotly.express as px

# Jupyter Magic
# %load_ext autoreload
# %autoreload 2
%matplotlib inline

In [2]:
# Setting the pyecharts config
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

# setting pyecharts' charts
from pyecharts.charts import Funnel, Bar, Line
from pyecharts import options as opts

In [3]:
# bar = (
#     Bar()
#     .add_xaxis([2,2])
#     .add_yaxis("e", [1,14])
# )
# bar.load_javascript()

In [4]:
# bar.render_notebook()

In [5]:
df = pd.read_csv('../Data/UserBehavior.csv', nrows=1000000, header=None, parse_dates=True)

In [6]:
df.columns = ['User_Id', 'Item_Id', 'Category_Id', 'Behavior_Type', 'Time']

In [7]:
# trans the dtype
df['Time'] = pd.to_datetime(df['Time'], unit='s')
df['Time'] = df.Time.astype('str')
df.head()

,User_Id,Item_Id,Category_Id,Behavior_Type,Time
0,1,2268318,2520377,pv,2017-11-24 17:21:10
1,1,2333346,2520771,pv,2017-11-24 22:15:33
2,1,2576651,149192,pv,2017-11-25 01:21:25
3,1,3830808,4181361,pv,2017-11-25 07:04:53
4,1,4365585,2520377,pv,2017-11-25 07:49:06


In [8]:
df['Time_Split'] = df.Time.str.split(' ')
df['Date'] = df.Time_Split.str.get(0)
df['Hour'] = df.Time_Split.str.get(1)

# Drop the Column
df = df.drop(['Time_Split'], axis=1)

In [9]:
# Redefine the order of columns 
df = df[['Date', 'Hour', 'Time', 'User_Id', 'Item_Id', 'Category_Id', 'Behavior_Type']]
df.head()

,Date,Hour,Time,User_Id,Item_Id,Category_Id,Behavior_Type
0,2017-11-24,17:21:10,2017-11-24 17:21:10,1,2268318,2520377,pv
1,2017-11-24,22:15:33,2017-11-24 22:15:33,1,2333346,2520771,pv
2,2017-11-25,01:21:25,2017-11-25 01:21:25,1,2576651,149192,pv
3,2017-11-25,07:04:53,2017-11-25 07:04:53,1,3830808,4181361,pv
4,2017-11-25,07:49:06,2017-11-25 07:49:06,1,4365585,2520377,pv


In [10]:
# Tranform the dtype back to datetime, in order to split the "Date" Column to two columns.
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 7 columns):
Date             1000000 non-null datetime64[ns]
Hour             1000000 non-null object
Time             1000000 non-null datetime64[ns]
User_Id          1000000 non-null int64
Item_Id          1000000 non-null int64
Category_Id      1000000 non-null int64
Behavior_Type    1000000 non-null object
dtypes: datetime64[ns](2), int64(3), object(2)
memory usage: 53.4+ MB


In [11]:
# Clean the wrong data range. Ex. outer the range from 2017,11,23 to 2017,12,24.
mask_1 = df['Date'] >= pd.datetime(2017,11,24)
mask_2 = df['Date'] <= pd.datetime(2017,12,24)
df = df[mask_1 & mask_2]

---
# Alibaba User Behavior 分析報告：

## A. 問題挖掘
* 利用電商常用模型 AARRR 與 RFM ，來分析各階段的留客率並挖掘洞見，找出能夠改善的環節並優化。
* 利用時間規律，找出用戶的活躍傾向，並藉此規劃行銷企劃。
* 分析用戶對不同商品的偏好程度，進一步設計各個商品的行銷計畫。
* 找到核心的付費用戶群，並據此進行用戶畫像的繪製。

## B. 資料概述
* 時間：2017 年 11 月 23 日至 2017 年 12 月 24 日
* 資料類別：
1. 用戶 ID
2. 物品 ID
3. 品類 ID
4. 行為類別
5. 時間戳記

## C. 分析細節

### 1. 分析參考模型
1. AARRR Model
2. RFM Model

### 2. AARRR｜漏斗模型分析
平均每個用戶在這 31 天的數據，共有 102.33 次的月瀏覽量，平均每日的瀏覽量共有 3.2 次。

#### i. Activation & Retention｜活躍度與留存分析
術語說明｜跳離率：僅一次點擊的用戶總數 / 用戶總量。

**結論**：淘寶擁有足夠的吸引力，讓用戶停留在 APP 中。
1. 資料統計為 32 天裡，983 個用戶中只有 55 個人瀏覽過一次便跳離，佔總訪問人數僅有 0.281%。
2. 平均每位用戶每天瀏覽淘寶 APP 10.17 次，說明用戶對淘寶服務的黏著度極高，基本為每日必使用的 APP。

#### ii. 用戶行為漏斗
**結論**：瀏覽到加入 購物車 / 我的最愛 環節明顯有提高的空間，是提升指標的重要環節。

因為放入購物車和加入我的最愛，都屬於用戶展示購買意願的階段，且因用戶習慣不同並無特別順序，因此將兩者皆算做同階段，並繪製漏斗圖展示轉換率。
1. 可以看到從瀏覽到購買僅有 9.13% 的轉化率，當然也會有部分用戶是跳過加入最愛直接購買。
2. 從我的最愛與購物車中轉化至購買的比例則有 25.65%，其中加入購物車（Cart）的比例為 66.5% 明顯高於我的最愛（Fav），顯示多數用戶多會直接加入購物車。
</b></b>

#### iii. 用戶行為分析解構：（後面再來分析）
1. 瀏覽 -> 加入購物車 -> 購買  
2. 瀏覽 -> 購買
3. 瀏覽 -> 加我的最愛 -> 加入購物車 -> 購買
4. 瀏覽 -> 加入購物車 -> 購買

In [12]:
# Setting the variables
user_count = df.User_Id.nunique()
page_view = len(df['Behavior_Type'] == 'pv')
total_day_count = df.Date.nunique()
view_per_person = round(page_view/user_count, 2)
only_one_click = df.groupby(['User_Id'])['Behavior_Type'].count().iloc[:1][1]

# Analysis result OUTPUT
print('Total Days: {}'.format(total_day_count))
print('User Number: {} people'.format(user_count))
print('Page View: {}'.format(page_view))
# print('Page View / User (One Month): {} times'.format(view_per_person))
print('Page View / User: {} times (One Day)'.format(round(view_per_person/total_day_count, 2)))
print('Leaving Rate: {:.3f}%'.format(only_one_click/user_count * 100))

# Display the order of each stage of customer behaviors
df.Behavior_Type.value_counts(ascending=False)

Total Days: 10
User Number: 9739 people
Page View: 999904
Page View / User: 10.27 times (One Day)
Leaving Rate: 0.565%


pv      896010
cart     55447
fav      28088
buy      20359
Name: Behavior_Type, dtype: int64

#### iii. 獨立訪客用戶漏斗模型
可以看到使用用戶中有 68.47% 的用戶轉換為付費用戶，用戶的付費轉換率相當高。

In [13]:
# Setting each stage
stage = ['Pay', 'Fav + Cart', 'PV']
behavior_type_list = df.Behavior_Type.value_counts().sort_values(ascending=True).tolist()
user_behavior_funnel = [behavior_type_list[0], (behavior_type_list[1]+behavior_type_list[2]), behavior_type_list[3]]

# setting the range of funnel
y_pos = np.arange(len(user_behavior_funnel))

# Setting the funnel charts
funnel = (
    Funnel(init_opts = opts.InitOpts(width="500px", height="400px"))
    .add('分類', [list(z) for z in zip(stage, user_behavior_funnel)])
    .set_global_opts(title_opts=opts.TitleOpts(title="用戶漏斗"))
)

# Setting the pyecharts
funnel.load_javascript()

# Display the Conversion Rate
print('----- Conversion rate of each stage -----')
print('PV -> Fav + Cart: {:.2f}%'.format(user_behavior_funnel[1] / user_behavior_funnel[2] * 100))
print('Fav + Cart -> Pay: {:.2f}%'.format(user_behavior_funnel[0] / user_behavior_funnel[1] * 100))

----- Conversion rate of each stage -----
PV -> Fav + Cart: 9.32%
Fav + Cart -> Pay: 24.37%


In [14]:
# Display the funnel chart of pyecharts
funnel.render_notebook()

In [15]:
stage_detail = ['pv', 'fav', 'cart', 'buy']

def unique_user():
    '''
    Define each stage in a list
    '''
    user_count = []
    
    # append each unique user into the list
    for stage in stage_detail:    
        unique = df[df['Behavior_Type'] == stage]['User_Id'].nunique()
        user_count.append(unique)
    
    return user_count

# Plotting the Bar chart of the number of users
user_count_bar = (
    Bar(
        init_opts = opts.InitOpts(width="500px", height="400px", bg_color="white")
#         item_style_opts = opts.ItemStyleOpts(color="000000")
    )
    .add_xaxis(stage_detail)
    .add_yaxis('人數', [list(z) for z in zip(stage_detail, unique_user())])
    .set_global_opts(title_opts=opts.TitleOpts(title="獨立用戶數"))
)

# Display the bar chart
user_count_bar.load_javascript()

# print the conversion rate
print('淘寶的付費滲透率: {:.2f}%'.format(unique_user()[3] / unique_user()[0] * 100))

淘寶的付費滲透率: 68.92%


In [16]:
user_count_bar.render_notebook()

### 3. 不同時間尺度下用戶傾向
**分析主軸**：此部分嘗試以不同的時間尺度下，分析用戶的行為模式與活躍規律。

#### i. 分析一個月中用戶行為規律
此處以月為週期觀察，可以發現

In [17]:
# Extract the date to list
df_date_range = df.Date.dt.date.astype('str').unique().tolist()
df_date_range[:5]

['2017-11-24', '2017-11-25', '2017-11-26', '2017-11-27', '2017-11-28']

In [18]:
# Calculate the pivot table
attr = ['Date', 'Buy', 'Cart', 'Fav', 'Pv']
pivot_Category = pd.pivot_table(df, index='Date', columns="Behavior_Type", aggfunc=['count']).iloc[:, :4]

pivot_Category = pivot_Category.reset_index()
pivot_Category.columns = attr
pivot_Category

,Date,Buy,Cart,Fav,Pv
0,2017-11-24,188,662,354,11330
1,2017-11-25,1938,5782,2781,94656
2,2017-11-26,2122,5819,3041,94037
3,2017-11-27,2191,5381,2848,87328
4,2017-11-28,2189,5541,2819,88913
5,2017-11-29,2300,5561,3020,91743
6,2017-11-30,2292,5704,2864,94533
7,2017-12-01,2193,6375,3043,100934
8,2017-12-02,2549,7842,3915,125130
9,2017-12-03,2397,6780,3403,107406


In [56]:
# Extract the column to the list
user = []
for stage in attr:
    user.append(pivot_Category[stage].astype('str').to_list())

    
def monthly_user_behavior():
    '''
    Create the monthly chart to analyze the user behavior in one month.
    '''
    user_bar = (
        Bar()
        .add_xaxis(user[0])
        .add_yaxis('購買人數', user[1], stack='stack1', category_gap=40)
        .add_yaxis('放入購物車', user[2], stack='stack1', category_gap=40)
        .add_yaxis('我的最愛', user[3], stack='stack1', category_gap=40)
        .extend_axis(
                yaxis=opts.AxisOpts(
                    axislabel_opts=opts.LabelOpts(formatter="{value} 人"), interval=20000
                )
            )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="用戶變化週期曲線"),
            yaxis_opts=opts.AxisOpts(
                axislabel_opts=opts.LabelOpts(formatter="{value} 人"), interval=20000
            ),
#             datazoom_opts=opts.DataZoomOpts(orient="vertical"),
        )
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    )

    user_line = (
        Line()
        .add_xaxis(user[0])
        .add_yaxis('購買人數', user[1], stack='stack2', yaxis_index=1)
        .add_yaxis('放入購物車', user[2], stack='stack2', yaxis_index=1)
        .add_yaxis('我的最愛', user[3], stack='stack2', yaxis_index=1)
        .add_yaxis('購買', user[4], stack='stack2', yaxis_index=1)
    )
    # overlap two charts
    user_bar.overlap(user_line)
    
    #return the chart
    return user_bar

# user_bar.render_notebook()
monthly_user_behavior().render_notebook()

In [54]:
def overlap_bar_line() -> Bar:
    bar = (
        Bar()
        .add_xaxis(user[0])
        .add_yaxis('購買人數', user[1], stack='stack1', category_gap=40)
        .add_yaxis('放入購物車', user[2], stack='stack1', category_gap=40)
        .add_yaxis('我的最愛', user[3], stack='stack1', category_gap=40)
        .extend_axis(
            yaxis=opts.AxisOpts(
                axislabel_opts=opts.LabelOpts(formatter="{value} 人"), interval=2000
            )
        )
        .set_series_opts(label_opts = opts.LabelOpts(is_show = False))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="用戶變化週期曲線")
        )
    )

    line = Line().add_xaxis(user[0]).add_yaxis('瀏覽人數', user[4], yaxis_index=1)
    line.overlap(bar)
    
    return bar

overlap_bar_line().render_notebook()

#### ii. 分析一週中用戶行為規律
時間：2017-11-27 (Monday) ~ 2017-12-03 (Sunday)
我們取雙十二前一週進行參考，可以發現禮拜四為各指標最低的一天，隨後持續上漲至禮拜六為高峰。推測是禮拜五開始上班族下班後開始頻繁使用淘寶，而週六有充足的時間與精神進行購買行為。
對於瀏覽量，可以看見週一瀏覽量明顯較低，且禮拜日的我的最愛明顯較高。因此若要維持用戶活躍，能於禮拜一進行少量推播，並根據用戶所加入的我的最愛，進行個性化推播，應能吸引用戶維持黏著度。

##### **建議**：
* 平時可以把行銷活動重點資源，投注在禮拜五至假日。
* 於週一進行少量推播，並根據用戶的我的最愛進行個性化推播，維持用戶活躍習慣。


In [21]:
# Mapping the day of week
map_weekday = {
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
}
# Mapping It
day_of_week = pd.to_datetime(user[0]).to_series().dt.dayofweek.map(map_weekday)
day_of_week

2017-11-24       Friday
2017-11-25     Saturday
2017-11-26       Sunday
2017-11-27       Monday
2017-11-28      Tuesday
2017-11-29    Wednesday
2017-11-30     Thursday
2017-12-01       Friday
2017-12-02     Saturday
2017-12-03       Sunday
dtype: object

In [22]:
def A_week_of_behavior():
    week_data = pivot_Category.iloc[3:]
    week_day = day_of_week[3:7].to_list()
    bar = (
        Bar()
        .add_xaxis(week_day)
        .add_yaxis('購買人數', [list(z) for z in zip(week_day, week_data['Buy'])], stack='stack2', category_gap=100)
        .add_yaxis('我的最愛', [list(z) for z in zip(week_day, week_data['Fav'])], stack='stack2', category_gap=100)
        .add_yaxis('購物車', [list(z) for z in zip(week_day, week_data['Cart'])], stack='stack2', category_gap=100)
        .extend_axis(
            yaxis=opts.AxisOpts(
                axislabel_opts=opts.LabelOpts(formatter="{value} 人"), interval=20000
            )
        )
        .set_series_opts(label_opts = opts.LabelOpts(is_show = False))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="用戶變化週期曲線(一週變化)")
        )
    )
    
    line = Line().add_xaxis(week_day).add_yaxis('瀏覽人數', week_data['Pv'], yaxis_index=1)
    bar.overlap(line)
    
    return bar

A_week_of_behavior().render_notebook()

In [97]:
one_day

,Hour,Behavior_Type,Hours
41,17,pv,"[17, 30, 35]"
42,17,pv,"[17, 38, 49]"
43,23,pv,"[23, 58, 52]"
145,10,pv,"[10, 13, 39]"
146,10,pv,"[10, 15, 40]"
...,...,...,...
999626,15,fav,"[15, 35, 18]"
999627,15,pv,"[15, 35, 30]"
999628,15,fav,"[15, 35, 54]"
999629,15,pv,"[15, 36, 00]"


In [198]:
# split the day of 12/01 into hour
def one_day_func():
    one_day = df[df['Date'] == '2017-12-01'][['Hour', 'Behavior_Type']]
    one_day['Hours'] = one_day.Hour.str.split(':')
    one_day['Hour'] = one_day.Hour.str[:2]
    final_df = one_day.drop(columns='Hours')
    final_df.columns = ['Hour', 'Be_Type']
    hour = final_df.groupby(['Be_Type'])['Hour'].count()
    return hour

one_day_func()

Be_Type
buy       2193
cart      6375
fav       3043
pv      100934
Name: Hour, dtype: int64

In [148]:
# # split the day of 12/01 into hour
# one_day = df[df['Date'] == '2017-12-01'][['Hour', 'Behavior_Type']]
# one_day['Hours'] = one_day.Hour.str.split(':')
# one_day['Hour'] = one_day.Hour.str[:2]

# def day_of_behavior():
#     # setting the number
#     hour_behav = one_day.groupby(['Hour', 'Behavior_Type']).count().reset_index()
    
#     hour_list = np.arange(1,24,1).tolist()
    
#     line = (
#         Line()
#         .add_xaxis(hour_list)
#         .add_yaxis("人數", [list(z) for z in zip(hour_list, hour_behav[hour_behav['Hour'] == "00"]))
#     )
    
#     return hour_behav
# day_of_behavior()
# # .render_notebook()

KeyError: 'buy'

In [166]:
hour_behav = pd.pivot_table(one_day, index="Hour", columns="Behavior_Type", aggfunc=['count'])
hour_behav = one_day.groupby('Hour')['Behavior_Type'].count()

o = one_day.groupby(['Hour', 'Behavior_Type']).count().reset_index()
o

,Hour,Behavior_Type,Hours
0,00,buy,65
1,00,cart,206
2,00,fav,116
3,00,pv,3259
4,01,buy,104
...,...,...,...
91,22,pv,1197
92,23,buy,41
93,23,cart,221
94,23,fav,108


#### iii.一天內的變化趨勢

這個部分如果參考上方，應該能夠在一個小時內完成。

# D.不同商品種類的用戶行為
統計所有被購買的商品，可以發現單項商品最高僅被購買 4 次，沒有特別集中的銷售商品，說明商品銷售多依靠長尾效應銷售，而非特殊熱門商品帶動營收。\

下方表格顯示出

In [208]:
items_count = df.query('Behavior_Type == "buy"').groupby('Item_Id')['Item_Id'].count()
items_count = items_count.sort_values(ascending=False)['']
items_top_twenty = items_count[:20].to_list()
items_view = df.query('Behavior_Type == "pv"').groupby('Item_Id')['Item_Id'].count()

#COunt first twenty items
items_index = items_count.index.to_list()[:20]

In [207]:
def item_top_twenty():
    
    bar = (
        Bar()
        .add_xaxis(items_index)
        .add_yaxis('購買數量', items_top_twenty, color="gray")
        .add_yaxis('瀏覽數量', )
        .set_global_opts(title_opts=opts.TitleOpts(title="商品銷售排行"))
        .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
    )
    return bar

item_top_twenty().render_notebook()

In [ ]:
line = Line().add_xaxis([1]).add_yaxis('Test',[5])
line.load_javascript()

In [ ]:
line.render_notebook()

## 參考 seaborn 來繪製商品排序圖，簡單一點就用橫條圖就好，不要炫技浪費時間。

同時參考瀏覽次數來進行比較：
* 比較方法，找出前面購買次數最多的那幾項商品，同時 plotting 出相對應的曲線圖（用 overlap 顯示相對應的瀏覽次數）。
* 單純顯示前面瀏覽次數最多的商品。
* 寫出商品的販售並不能以瀏覽次數作為參考，同時商品也並非有特定熱消款式，而是根據常尾效益來提升營收。